pip install efficientnet_pytorch

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize, RandomVerticalFlip
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

from PIL import Image
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from efficientnet_pytorch import EfficientNet

In [3]:
TRAIN_PATH = '/opt/ml/input/data/train/'
TRAIN_IMAGE_PATH = '/opt/ml/input/data/train/processed_train_images/'

In [4]:
train_df = pd.read_csv(os.path.join(TRAIN_PATH, 'preprocessed_train.csv'))
train_df.sample(10)

,Unnamed: 0,id,gender,mask,age,age_3,age_11,label,path
9640,9640,004371,1,0,56,1,5,4,/opt/ml/input/data/train/processed_train_image...
13259,13259,006231,0,0,20,0,2,0,/opt/ml/input/data/train/processed_train_image...
6014,6014,006728,0,0,19,0,1,0,/opt/ml/input/data/train/processed_train_image...
12101,12101,006652,0,0,20,0,2,0,/opt/ml/input/data/train/processed_train_image...
10144,10144,003617,0,0,56,1,5,1,/opt/ml/input/data/train/processed_train_image...
2620,2620,001573,1,2,34,1,3,16,/opt/ml/input/data/train/processed_train_image...
9269,9269,001407,0,0,43,1,4,1,/opt/ml/input/data/train/processed_train_image...
12071,12071,004246,1,1,38,1,3,10,/opt/ml/input/data/train/processed_train_image...
17012,17012,003669,0,0,59,1,5,1,/opt/ml/input/data/train/processed_train_image...
16308,16308,003725,0,2,58,1,5,13,/opt/ml/input/data/train/processed_train_image...


In [5]:
class maskDataset(Dataset):
    def __init__(self, train_df, transform, train=True):
        self.train = train
        self.transform = transform

        if self.train == True:
            self.paths = train_df['path']
            self.labels = train_df['age_3']
        else:
            self.paths = train_df
        

    def __getitem__(self, index):
        if self.train:
            image = Image.open(self.paths.iloc[index])
            label = self.labels.iloc[index]
        else:
            image = Image.open(self.paths[index])

        if self.transform:
            image = self.transform(image)

        if self.train:
            return image, torch.tensor(label)
        else:
            return image

    def __len__(self):
        return len(self.paths)

In [6]:
transform = transforms.Compose([
    # Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    # Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
    RandomVerticalFlip()
])

In [7]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_df, test_size=0.2, shuffle=True, stratify=train_df['label'], random_state=42)

In [8]:
train.shape, valid.shape

((15114, 9), (3779, 9))

In [9]:
BATCH_SIZE = 32

In [10]:
train_dataset = maskDataset(train, transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

In [11]:
valid_dataset = maskDataset(valid, transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

In [12]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 3, 512, 384])

In [13]:
class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super(EfficientNet_MultiLabel, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.network = EfficientNet.from_pretrained('efficientnet-b0', in_channels=self.in_channels, num_classes=self.num_classes)

    def forward(self, x):
        return self.network(x)

efficientNet = EfficientNet_MultiLabel()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ('My Device :', device)
efficientNet = efficientNet.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0
My Device : cuda:0


In [14]:
LEARNING_RATE = 1e-4
NUM_EPOCH = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(efficientNet.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_dataloader,
    "valid" : valid_dataloader
}

In [15]:
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=20)

In [16]:
best_acc = 0
best_loss = 0

PRINT_EVERY = 10

for epoch in tqdm(range(NUM_EPOCH)):
    for phase in ['train', 'valid']:
        running_loss = 0
        running_acc = 0

        if phase == 'train':
            efficientNet.train()
        elif phase == 'valid':
            efficientNet.eval()

        for idx, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                logits = efficientNet(images)
                _, pred = torch.max(logits, 1)
                loss = loss_fn(logits, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(pred == labels.data)

            # if idx % PRINT_EVERY == 0:
            #     train_loss = running_loss / PRINT_EVERY
            #     train_acc = running_acc / BATCH_SIZE / PRINT_EVERY
            #     print (f"Epoch={epoch}, Training Loss={train_loss:4.4}, Training Accuracy={train_acc:4.2%}, lr={LEARNING_RATE}")
            
        scheduler.step()

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print (f'[{phase}] 현재 EPOCH : {epoch}, loss : {epoch_loss}, acc : {epoch_acc}')


[train] 현재 EPOCH : 0, loss : 0.282469979832061, acc : 0.8987031579017639
[valid] 현재 EPOCH : 0, loss : 0.14091086961756816, acc : 0.9396665692329407
[train] 현재 EPOCH : 1, loss : 0.10675641427607042, acc : 0.9602354764938354
[valid] 현재 EPOCH : 1, loss : 0.0587857291753029, acc : 0.9780364632606506
[train] 현재 EPOCH : 2, loss : 0.04933021511856417, acc : 0.9836574792861938
[valid] 현재 EPOCH : 2, loss : 0.08215932732782656, acc : 0.9714210033416748
[train] 현재 EPOCH : 3, loss : 0.03127837554236537, acc : 0.989281415939331
[valid] 현재 EPOCH : 3, loss : 0.024553267296317884, acc : 0.9902090430259705
[train] 현재 EPOCH : 4, loss : 0.0185572738305451, acc : 0.9938467144966125
[valid] 현재 EPOCH : 4, loss : 0.02856280219283514, acc : 0.9867689609527588
[train] 현재 EPOCH : 5, loss : 0.01318413883894884, acc : 0.9953023195266724
[valid] 현재 EPOCH : 5, loss : 0.019652650114359568, acc : 0.9923259615898132
[train] 현재 EPOCH : 6, loss : 0.009475189416543729, acc : 0.9966256022453308
[valid] 현재 EPOCH : 6, loss 

In [17]:
TEST_DIR = '/opt/ml/input/data/eval/'

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다
submission = pd.read_csv(os.path.join(TEST_DIR, 'info.csv'))
image_dir = os.path.join(TEST_DIR, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]

transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = maskDataset(image_paths, transform, train=False)
loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
ResNet18.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    images = images.to(device)
    with torch.no_grad():
        logits = ResNet18(images)
        _, pred = torch.max(logits, 1)
        # pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(TEST_DIR, 'submission.csv'), index=False)
print('test inference is done!')